## Importing needed packages

In [95]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

## Defining Input Data

In [96]:
# Create a dataframe for Capacity Cost
CapCost = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'CapCost[€/MW]': [500, 550, 2500, 150, 300]
})

# Create a dataframe for Operating Cost
OpCost = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'OpCost[€/MWh]': [200, 250, 1550, 0, 0]
})

# Create a dataframe for Storage Cost
StorCost = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'StorCost[€/MWh]': [10, 500000]
})

# Create a dataframe for Capacity Limit
CapLim = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'CapLim[MW]': [100, 100, 100, 100, 100]
})

# Create a dataframe for existing capacity
CapExi = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'ExCap[MW]': [100, 50, 50, 100, 100]
})

# Create a dataframe for outphased capacity
CapOut = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'OutCap[MW]': [0, 0, 0, -50, 0]
})
# Create a dataframe for production factor on hourly basis
ProdFac = pd.DataFrame({
    'Coal': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'Gas': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'Nuclear': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'Wind': [0.7819, 0.9992, 0.4135, 0.1235, 0.7744, 0.4485, 0.0786, 0.5854, 0.7355, 0.0058, 0.9286, 0.3051, 0.4102, 0.0587, 0.1249, 0.2616, 0.0291, 0.3583, 0.2015, 0.8517, 0.8301, 0.2887, 0.4119, 0.0677],
    'Solar': [0.2328, 0.9621, 0.7316, 0.9729, 0.5756, 0.9492, 0.2877, 0.3097, 0.7812, 0.6488, 0.1969, 0.9525, 0.4616, 0.8437, 0.9113, 0.0780, 0.9759, 0.3342, 0.9149, 0.3129, 0.7933, 0.4664, 0.3363, 0.067]
})

# Create a dataframe for demand on hourly basis
Demand = pd.DataFrame({
    'Hour': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    'Demand[MWh]': [500,600,555,343,644,343,535,223,535,634,343,535,223,535,634,343,535,223,535,634,343,535,223,535]
})

# Create a dataframe for eta charge
EtaCh = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'EtaCh': [0.9, 0.8]
})

# Create a dataframe for eta discharge
EtaDis = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'EtaDis': [0.9, 0.8]
})

# Create a dataframe for existing storage capacity
StorExi = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'ExStor[MWh]': [40, 0]
})


## Defining Input Parameters

In [97]:
# Define number of hours as length of demand
N_Hours = len(Demand)

# Define number of generators as length of CapCost
N_Cap = len(CapCost)

# Define number of storage as length of StorCost
N_Stor = len(StorCost)

# Define epsilon
epsilon = 0.9

#Define delta
delta = 0.8

## Defining classes

In [98]:
# Class for external input data
class InputData():
    def __init__(self, CapCost, OpCost, StorCost, CapLim,CapExi,CapOut,ProdFactor,Dem,EtaCha,EtaDis,StorExi):
        self.CapCost = CapCost
        self.OpCost = OpCost
        self.StorCost = StorCost
        self.CapLim = CapLim
        self.CapExi = CapExi
        self.CapOut = CapOut
        self.ProdFactor = ProdFactor
        self.Dem = Dem
        self.EtaCha = EtaCha
        self.EtaDis = EtaDis
        self.StorExi = StorExi

        

In [99]:
# Class for model parameters
class Parameters():
    def __init__(self, epsilon, delta, N_Hours, N_Cap, N_Stor):
        self.epsilon = epsilon
        self.delta = delta
        self.N_Hours = N_Hours
        self.N_Cap = N_Cap
        self.N_Stor = N_Stor

## Creating Data and Parameter Objects

In [100]:
ParametersObj = Parameters(epsilon, delta, N_Hours, N_Cap, N_Stor)
DataObj = InputData(CapCost, OpCost, StorCost, CapLim,CapExi,CapOut,ProdFac,Demand,EtaCh,EtaDis,StorExi)


## Creating the Model

In [101]:
# CLASS WHICH CAN HAVE ATTRIBUTES SET

class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [102]:
# Defining the optimization model class

class CapacityProblem():
    def __init__(self, ParametersObj, DataObj, Model_results = 1, Guroby_results = 1):
        self.P = ParametersObj # Parameters
        self.D = DataObj # Data
        self.Model_results = Model_results
        self.Guroby_results = Guroby_results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 


    def _build_variables(self):
        # Create the variables
        self.var.CapNew = self.m.addMVar((self.P.N_Cap), lb=0)  # New Capacity for each type of generator technology
        self.var.EGen = self.m.addMVar((self.P.N_Cap, self.P.N_Hours), lb=0)  # Energy Production for each type of generator technology for each hour and scenario
        self.var.CapStor = self.m.addMVar((self.P.N_Stor), lb=0)  # New storage capacity for each type of storage technology
        self.var.SOC = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # State of charge for each type of storage technology for each hour and scenario  
        self.var.EChar = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # Energy charged for each type of storage technology for each hour and scenario
        self.var.EDis = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # Energy discharged for each type of storage technology for each hour and scenario


    def _build_constraints(self):
        # Limit new capacity by maximum investable capacity for each type of generator technology
        for g in range(self.P.N_Cap):
            self.con.CapLim = self.m.addConstr(self.var.CapNew[g] <= self.D.CapLim['CapLim[MW]'][g], name='Capacity limit')
        
        # Production limited by sum of new capacity, existing capacity and phased out capacity times the production factor
        for g in range(self.P.N_Cap):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.ProdLim = self.m.addConstr(self.var.EGen[g, h] <= (self.var.CapNew[g] + self.D.CapExi['ExCap[MW]'][g] + self.D.CapOut['OutCap[MW]'][g]) * self.D.ProdFactor.iloc[h,g], name='Production limit')

        # Energy Balance constraint - generation and discharge needs to equal demand and charging for each hour and scenario
        for h in range(self.P.N_Hours):
            #for s in range(self.P.N_Scen):
                self.con.Balance = self.m.addConstr(gp.quicksum(self.var.EGen[:, h]) + gp.quicksum(self.var.EDis[:, h]) == self.D.Dem['Demand[MWh]'][h] + gp.quicksum(self.var.EChar[:, h]), name='Energy balance')

        # Defining RES share as a percentage of total energy demand for each scenario
        # for s in range(self.P.N_Scen):
        #self.con.RESShare = self.m.addConstr(gp.quicksum(self.var.EGen[0, :, s]) >= self.P.epsilon * gp.quicksum(self.D.Dem[:, s]), name='RES share')

        # Defining SOC as SOC from previous hour plus energy charged minus energy discharged for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(1, self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.SOC = self.m.addConstr(self.var.SOC[u, h] == self.var.SOC[u, h-1] + self.var.EChar[u, h] - self.var.EDis[u, h], name='State of charge')

        # Define SOC for first hour
        for u in range(self.P.N_Stor):
            #for s in range(self.P.N_Scen):
                self.con.SOC0 = self.m.addConstr(self.var.SOC[u, 0] == (self.D.StorExi['ExStor[MWh]'][u])*0, name='State of charge 0')

        # Limit SOC by maximum storage capacity for each type of storage technology
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.SOCLim = self.m.addConstr(self.var.SOC[u, h] <= self.var.CapStor[u] + self.D.StorExi['ExStor[MWh]'][u], name='Storage capacity limit')

        # Limit charged energy by capacity minus state of charge for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.ECharLim = self.m.addConstr(self.var.EChar[u, h] <= self.var.CapStor[u] + self.D.StorExi['ExStor[MWh]'][u] - self.var.SOC[u, h], name='Energy charged limit')

        # Limit discharged energy by state of charge for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.EDisLim = self.m.addConstr(self.var.EDis[u, h] <= self.var.SOC[u, h], name='Energy discharged limit')


        
    
    def _build_objective(self):
        # Objective function
        objective = gp.quicksum(self.var.CapNew[g] * self.D.CapCost['CapCost[€/MW]'][g] for g in range(self.P.N_Cap))  # Cost for new capacity for each type of generator technology
        + gp.quicksum(self.var.EGen[g,h] * self.D.OpCost['OpCost[€/MWh]'][g] for g in range(self.P.N_Cap) for h in range(self.P.N_Hours)) # Operating cost for each type of generator technology for each hour and scenario
        + gp.quicksum(self.var.CapStor[u] * self.D.StorCost['StorCost[€/MWh]'][u] for u in range(self.P.N_Stor)) # Cost for new storage capacity for each type of storage technology
        self.m.setObjective(objective, GRB.MINIMIZE)


    def _display_guropby_results(self):
        self.m.setParam('OutputFlag', self.Guroby_results)

    
    def _build_model(self):
        self.m = gp.Model('CapacityProblem')
        self._build_variables()
        self._build_constraints()
        self._build_objective()
        self._display_guropby_results()
        self.m.optimize()
        self._results()
        if self.Model_results == 1:
            self._extract_results()
            

    
    def _results(self):
        self.res.obj = self.m.objVal
        self.res.CapNew = self.var.CapNew.X
        self.res.EGen = self.var.EGen.X
        self.res.CapStor = self.var.CapStor.X
        self.res.SOC = self.var.SOC.X
        self.res.EChar = self.var.EChar.X
        self.res.EDis = self.var.EDis.X
        
        #Create Dataframe for new capacity
        self.res.CapNew = pd.DataFrame({
            'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
            'CapNew[MW]': self.res.CapNew
        })

        #Create Dataframe for new storage capacity
        self.res.CapStor = pd.DataFrame({
            'Plant': ['Battery', 'Pumped Hydro'],
            'CapStor[MWh]': self.res.CapStor
        })

        #Create one Dataframe for demand, energy production, state of charge, energy charged and energy discharged
        self.res.Demand = self.D.Dem
        self.res.EGen = pd.DataFrame(self.res.EGen.T, columns = ['Coal Gen', 'Gas Gen', 'Nuclear Gen', 'Wind Gen', 'Solar Gen'])  
        self.res.SOC = pd.DataFrame(self.res.SOC.T, columns = ['Battery SOC', 'Pumped Hydro SOC'])
        self.res.EChar = pd.DataFrame(self.res.EChar.T, columns = ['Battery Char', 'Pumped Hydro Char'])
        self.res.EDis = pd.DataFrame(self.res.EDis.T, columns = ['Battery Dis', 'Pumped Hydro Dis'])

        #Combine all dataframes into one
        self.res.All = pd.concat([self.res.Demand, self.res.EGen, self.res.SOC, self.res.EChar, self.res.EDis], axis=1)
        
        

        
    

    def _extract_results(self):
        # Display the objective value
        print('Objective value: ', self.m.objVal)
        # Display the new capacity for each type of generator technology
        print('New capacity for each type of generator technology: ', self.res.CapNew)
        
        #Display new storage capacity for each type of storage technology
        print('New storage capacity for each type of storage technology: ', self.res.CapStor)

        

## Execute the model

In [103]:
CapacityProblem = CapacityProblem(ParametersObj, DataObj)
#CapacityProblem._results()
#CapacityProblem._extract_results()

ResultsCapacity = CapacityProblem.res.CapNew
AllResults = CapacityProblem.res.All



Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 341 rows, 271 columns and 983 nonzeros
Model fingerprint: 0xc594e678
Coefficient statistics:
  Matrix range     [6e-03, 1e+00]
  Objective range  [2e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 6e+02]
Presolve removed 152 rows and 57 columns
Presolve time: 0.00s
Presolved: 189 rows, 214 columns, 856 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.813625e+03   0.000000e+00      0s
     189    1.0916404e+05   0.000000e+00   0.000000e+00      0s

Solved in 189 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.091640375e+05
Objective value:  109164.03753836355
New capacity for each type of generator technology:       Plant  Cap

In [104]:
CapacityProblem.res.EGen

,Coal Gen,Gas Gen,Nuclear Gen,Wind Gen,Solar Gen
0,200.0,93.900396,50.000000,117.285000,38.814604
1,200.0,93.900396,50.000000,149.880000,160.410355
2,200.0,93.900396,50.000000,62.025000,121.979229
3,200.0,93.900396,50.000000,18.525000,162.211033
4,200.0,93.900396,50.000000,116.160000,95.969442
5,200.0,93.900396,50.000000,67.275000,158.259546
6,200.0,93.900396,50.000000,11.790000,47.968048
7,200.0,93.900396,0.000000,25.881313,0.000000
8,200.0,93.900396,50.000000,110.325000,130.249007
9,200.0,93.900396,50.000000,0.870000,108.174034
